# Import libraries

In [1]:
import os
import pandas as pd
from lxml import etree
! pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


# Connect to the shareddrive folder for this project

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
shareddrive_path = '/content/drive/Shareddrives/MACS_30122_Project'

In [4]:
os.chdir(shareddrive_path)

In [5]:
!ls

2018_nsf.csv				libu2f-udev_1.1.4-1_all.deb  NSF_zip
google-chrome-stable_current_amd64.deb	NSF_data


# Set up selenium configuration for Google Colab

In [6]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,678 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main Sources [2,268 kB]
Get:13 http://archive.ubuntu.com/ubuntu jamm

In [7]:
!pip install chromedriver-autoinstaller

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
import chromedriver_autoinstaller

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

'/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/120/chromedriver'

# Load NSF data


In [8]:
# Unzip the file

# Create the main directory to store OSF data (xml files) if it doesn't exist
if not os.path.isdir("NSF_data"):
    os.mkdir("NSF_data")

# Unzip the file into a subdirectory
if not os.path.isdir("NSF_data/2018"):
    !unzip "NSF_data/2018.zip" -d "NSF_data/2018"

This is a website introducing the directory and affiliated divisions: https://new.nsf.gov/about/directorates-offices

In [9]:
# Extracts data from downloaded NSF files
def extract_data_from_file(file_path):
    tree = etree.parse(file_path)
    root = tree.getroot()

    extracted_data = {
        "first_name": "",
        "middle_name": "",
        "last_name": "",
        "email": "",
        "directorate": "",
        "division": "",
        "effective_date": "",
        "expiration_date": "",
        "award_amount": "",
        "abstract": ""
    }

    award = root.find('Award')

    extracted_data["effective_date"] = award.findtext('AwardEffectiveDate') or ''
    extracted_data["expiration_date"] = award.findtext('AwardExpirationDate') or ''
    extracted_data["award_amount"] = award.findtext('AwardTotalIntnAmount') or ''
    extracted_data["abstract"] = award.findtext('AbstractNarration') or ''
    extracted_data["directorate"] = award.findtext('Organization/Directorate/LongName') or ''
    extracted_data["division"] = award.findtext('Organization/Division/LongName') or ''
    # I do not think we need abbreviation
    # extracted_data["division_abbreviation"] = award.findtext('Organization/Division/Abbreviation')
    extracted_data["first_name"] = award.findtext('Investigator/FirstName') or ''
    extracted_data["middle_name"] = award.findtext('Investigator/PI_MID_INIT') or ''
    extracted_data["last_name"] = award.findtext('Investigator/LastName') or ''
    extracted_data["email"] = award.findtext('Investigator/EmailAddress') or ''

    return extracted_data


# Processes files in a folder
def process_folder(folder_path, filter_directorate):
    all_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            file_path = os.path.join(folder_path, filename)
            data = extract_data_from_file(file_path)
            if data["directorate"] == filter_directorate:
              all_data.append(data)

    return pd.DataFrame(all_data)

In [10]:
# example use for extracting
folder_path = 'NSF_data/2018' # path to folder where NSF files are stored
file_path = '2018_nsf.csv'

if os.path.isfile(file_path):
  nsf_df = pd.read_csv(file_path)
else:
  nsf_df = process_folder(folder_path, filter_directorate="Direct For Social, Behav & Economic Scie") # process the folder
  nsf_df.to_csv('2018_nsf.csv', index=False) # save the dataframe in a csv file

In [ ]:
nsf_df.head()

,first_name,middle_name,last_name,email,directorate,division,effective_date,expiration_date,award_amount,abstract
0,Herschel,F,Thomas,herschel.thomas@mail.wvu.edu,"Direct For Social, Behav & Economic Scie",Divn Of Social and Economic Sciences,03/15/2018,10/31/2018,71473.0,Natural disasters place immense stress on gove...
1,Susan,C,Anton,susan.anton@nyu.edu,"Direct For Social, Behav & Economic Scie",Division Of Behavioral and Cognitive Sci,10/15/2017,11/30/2022,199080.0,Extreme and unexpected environmental changes h...
2,Thomas,J,Cova,cova@geog.utah.edu,"Direct For Social, Behav & Economic Scie",Divn Of Social and Economic Sciences,07/15/2018,06/30/2024,298625.0,This award will support the establishment of t...
3,Jeffrey,NaN,Hooker,hooker@cdkc.edu,"Direct For Social, Behav & Economic Scie",Division Of Behavioral and Cognitive Sci,02/15/2018,07/31/2023,40692.0,The state of Montana is home to nine Native Am...
4,Susan,NaN,Penfield,sdp@u.arizona.edu,"Direct For Social, Behav & Economic Scie",Division Of Behavioral and Cognitive Sci,02/15/2018,07/31/2024,49588.0,The state of Montana is home to nine Native Am...


In [11]:
nsf_df["division"].value_counts()

Division Of Behavioral and Cognitive Sci    481
Divn Of Social and Economic Sciences        388
SBE Off Of Multidisciplinary Activities      87
National Center For S&E Statistics            8
Name: division, dtype: int64

# Dynamically scrape the publishing-related info about the author on the NSF award list

## Define helper functions to dynamically scrape citation metrics, publication details, and research interests of a given author

In [12]:
# Finds Google Scholar urls
def find_url(driver, full_name, email_domain):
    url = f"https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors={full_name}"
    driver.get(url)
    time.sleep(3)

    authors = driver.find_elements(By.CSS_SELECTOR, "div.gs_ai.gs_scl.gs_ai_chpr")
    for author in authors:
        author_email_text = author.find_element(By.CSS_SELECTOR, "div.gs_ai_eml").text
        if 'Verified email at ' in author_email_text:
            author_email_domain = author_email_text.split('Verified email at ')[1]
            if email_domain == author_email_domain:
                link_element = author.find_element(By.CSS_SELECTOR, "a.gs_ai_pho")
                return link_element.get_attribute('href')

    return None


# Finds citations
def find_citations(driver, url):
    driver.set_window_size(800, 1000)
    driver.get(url)
    time.sleep(3)

    cited_by_tab = driver.find_element(By.ID, "gsc_prf_t-cit")
    cited_by_tab.click()
    time.sleep(3)

    total_citations = driver.find_element(By.XPATH, '//*[@id="gsc_rsb_st"]/tbody/tr[1]/td[2]').text
    h_index = driver.find_element(By.XPATH, '//*[@id="gsc_rsb_st"]/tbody/tr[2]/td[2]').text

    year_citations = {}
    year_elements = driver.find_elements(By.CSS_SELECTOR, "div.gsc_md_hist_w .gsc_g_t")
    citation_elements = driver.find_elements(By.CSS_SELECTOR, "div.gsc_md_hist_w .gsc_g_a")

    for year, citation in zip(year_elements, citation_elements):
        citation_count = driver.execute_script("return arguments[0].textContent", citation)
        year_citations[year.text] = citation_count

    return total_citations, h_index, year_citations


# Finds publications
# Here, I retreive not only the tile of the publication, but also its citation count, year of publication, and paper abstract
from selenium.webdriver.support.ui import WebDriverWait
def find_publications(driver, url):
    driver.get(url)
    time.sleep(3)

    while True:
        try:
            show_more_button = driver.find_element(By.ID, "gsc_bpf_more")
            if show_more_button.is_displayed() and show_more_button.is_enabled():
                show_more_button.click()
                time.sleep(2)
            else:
                break
        except (NoSuchElementException, ElementClickInterceptedException):
            break

    publications = []
    rows = driver.find_elements(By.CSS_SELECTOR, "tr.gsc_a_tr")

    for row in rows:
        # Extract title
        title_element = row.find_element(By.CSS_SELECTOR, "a.gsc_a_at")
        title = title_element.text

        # Extract coauthors
        coauthors_element = row.find_elements(By.CSS_SELECTOR, "td.gsc_a_t div.gs_gray")[0]
        coauthors = coauthors_element.text if coauthors_element else "Coauthors not found"

        # Extract citation count
        citation_element = row.find_element(By.CSS_SELECTOR, "td.gsc_a_c a.gsc_a_ac.gs_ibl")
        n_citation = citation_element.text if citation_element else "0"

        # Extract year of publication
        year_element = row.find_element(By.CSS_SELECTOR, "td.gsc_a_y span.gsc_a_h.gsc_a_hc.gs_ibl")
        year = year_element.text if year_element else "Year not found"

        # Navigate to the citation link page to extract abstract
        publication_url = title_element.get_attribute('href')
        driver.get(publication_url)
        time.sleep(3)  # Wait for the page to load

        # Extract the abstract
        try:
            abstract_element = driver.find_element(By.CSS_SELECTOR, "div.gsh_csp") # div.gsh_small
            abstract = abstract_element.text
        except NoSuchElementException:
            abstract = "Abstract not found"

        driver.back()

        publications.append({
            "title": title,
            "year": year,
            "coauthors": coauthors,
            "n_citation": n_citation
            "abstract": abstract
        })

    return publications, publication_url_lst

# Finds interests
def find_interests(driver, url):
    driver.get(url)
    time.sleep(3)

    interests = []

    try:
        interest = driver.find_elements(By.CSS_SELECTOR, "div#gsc_prf_int a.gsc_prf_inta")
        interests = [i.text for i in interest] if interest else None

    except Exception as e:
        print(f"Error occurred: {e}")

    return interests

## Define the function that scrapes publishing info of authors by relating to the `nsf_data` dataframe just created

In [13]:
# Cleans and updates the dataframe
from selenium.webdriver.chrome.service import Service as ChromeService

def update_and_save_dataframe(df, chrome_driver_path):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')  # Ensure GUI is off
    chrome_options.add_argument('--no-sandbox')  # Bypass OS security model
    chrome_options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems

    # Set up ChromeService using the specified path
    service = ChromeService(executable_path=chrome_driver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # Updates Google Scholar urls
    df = df.dropna(subset=['email'])
    df['url'] = None

    for index, row in df.iterrows():
        full_name = f"{row['first_name']} {row['middle_name']} {row['last_name']}".strip() if pd.notna(row['middle_name']) else f"{row['first_name']} {row['last_name']}"
        email_domain = row['email'].split('@')[-1]
        url = find_url(driver, full_name, email_domain)
        df.loc[index, 'url'] = url
        time.sleep(3)

    df = df.dropna(subset=['url'])
    df['publications'] = None
    df['interests'] = None

    for index, row in df.iterrows():
        url = row['url']

        # Updates citations
        total_citations, h_index, year_citations = find_citations(driver, url)
        df.at[index, 'total_citations'] = total_citations
        df.at[index, 'h_index'] = h_index

        for year, citations in year_citations.items():
            col_name = f'citations_{year}'
            df.at[index, col_name] = citations

        # Updates publications
        publications_dict, = find_publications(driver, url)
        df.at[index, 'publications'] = publications # For now I store the whole dictionary of title, n_citation, year, and abstract in a whole cell (which definitely could be improved in future)

        # Updates interests
        interests = find_interests(driver, url)
        if interests:
            df.at[index, 'interests'] = [interest for interest in interests]

        time.sleep(3)

    driver.quit()

    df.to_csv('2018_nsf_google_scholar.csv', index=False)

    return url_list


In [17]:
# path to chromedriver
chrome_driver_path = '/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/120/chromedriver'

In [18]:
# example use
update_and_save_dataframe(nsf_df[:1], chrome_driver_path) # updates and saves the dataframe into a new csv file

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome-headless-shell=120.0.6099.224); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
#0 0x5a46d8098f83 <unknown>
#1 0x5a46d7d51cf7 <unknown>
#2 0x5a46d7d63890 <unknown>
#3 0x5a46d7d58d54 <unknown>
#4 0x5a46d7d56bad <unknown>
#5 0x5a46d7d5a798 <unknown>
#6 0x5a46d7d5a8ec <unknown>
#7 0x5a46d7da16c7 <unknown>
#8 0x5a46d7da1aa1 <unknown>
#9 0x5a46d7d94c76 <unknown>
#10 0x5a46d7dcb0dd <unknown>
#11 0x5a46d7d94b49 <unknown>
#12 0x5a46d7dcb4ce <unknown>
#13 0x5a46d7dea006 <unknown>
#14 0x5a46d7dcae53 <unknown>
#15 0x5a46d7d92dd4 <unknown>
#16 0x5a46d7d941de <unknown>
#17 0x5a46d805d531 <unknown>
#18 0x5a46d8061455 <unknown>
#19 0x5a46d8049f55 <unknown>
#20 0x5a46d80620ef <unknown>
#21 0x5a46d802d99f <unknown>
#22 0x5a46d8086008 <unknown>
#23 0x5a46d80861d7 <unknown>
#24